In [1]:
import ipywidgets as widgets
from IPython.display import display
import plotly.graph_objects as go
import asyncio
import pandas as pd
import numpy as np
import datetime

import stream.bogiepandas as bogiepandas
import stream.stream as stream

SERVER="nats://192.168.23.159:4222"
EXPORT_SUBJECT = "bogie"
STREAM_NAME = "test"
NUM_SENSORS = 4


ModuleNotFoundError: No module named 'ipywidgets'

In [3]:

async def ns_from_time(t):
    stime = t.strftime("%Y-%m-%dT%H:%M:%S.%fZ")
    print("stime %s" % stime)
    stime="1990-01-01T00:00:00.000000Z"
    ns = await stream.NatsStream.from_start_time(
        SERVER, STREAM_NAME, EXPORT_SUBJECT, stime
    )
    # "1990-01-01T00:00:00.000000Z"
    return ns

async def ns_from_seq(seq):
    seq = int(seq)
    print("seq %d" % seq)
    ns = await stream.NatsStream.from_seq(
        SERVER, STREAM_NAME, EXPORT_SUBJECT, seq
    )
    return ns

async def nats_stream_fetch(ns, n):
    print("nats_stream_fetch %d" % n)
    df = pd.DataFrame()
    for i in range(n):
        msg = await ns.next_msg()
        if msg is None:
            print("msg is None")
            break
        await ns.ack(msg)
        df2 = bogiepandas.bogie_nats_to_pandas(msg)
        print("loaded time %s seq %d" % (df2.iloc[0]["nats_rx_time"], df2.iloc[0]["seq"]  ))
        df = pd.concat([df, df2], axis=0)
    return df

In [6]:
class DemoUI:
    def __init__(self, queue, df_cache_size=10):
        self.queue = queue
        self.df_cache_size = df_cache_size
        
        self.start_date_picker = widgets.DatePicker(
            description='Start Date',
            disabled=False
        )
        self.slider = widgets.IntSlider(
            min=0,
            max=9,
            step=1,
            readout=False, 
        )
        self.btn_left = widgets.Button(icon="chevron-left")
        self.btn_right = widgets.Button(icon="chevron-right")
        self.control_box = widgets.Box([self.start_date_picker, self.btn_left, self.slider, self.btn_right])

        self.sensor_wf_fig = go.FigureWidget()
        for i in range(NUM_SENSORS):
            self.sensor_wf_fig.add_trace(go.Scatter(x=[], y=[], name="sensor%d" % i))
        ax = go.layout.YAxis(
            fixedrange=True,
            range=[0, 1],
        )
        self.sensor_wf_fig.update_layout({"yaxis": ax})


        self.slider.observe(self.handle_slider_change, names='value')
        self.start_date_picker.observe(self.handle_start_date_change, names='value')
        self.btn_right.on_click(self.btn_right_eventhandler)
        self.btn_left.on_click(self.btn_left_eventhandler)

        vbox = widgets.VBox([self.control_box, self.sensor_wf_fig])

        self.df = pd.DataFrame()

        self.draw(0)
        display(vbox)

    def draw(self, idx):
        """
        draw dataframe row at idx
        """
        if idx >= self.df.shape[0]:
            return
        print("draw", idx)
        df = self.df.iloc[idx]
        fig = self.sensor_wf_fig
        
        fig.update_layout({"title": f"Dataset from {df['trigger_time']} seq {df['seq']}"})

        print("length of samples %d" % df['sensor_data'].shape[0])
        for i in range(NUM_SENSORS):
            fig.data[i].x = list(range(0,df['sensor_data'].shape[0]))
            fig.data[i].y = df["sensor_data"]["sensor%d" % i]


    async def load_from_time(self, time):
        ns = await ns_from_time(time)
        self.df = await nats_stream_fetch(ns, self.df_cache_size)
        self.slider.value = 0
        self.draw(0)

    async def load_from_seq(self, seq):
        ns = await ns_from_seq(seq)
        self.df = await nats_stream_fetch(ns, self.df_cache_size)
        self.slider.value = 0
        self.draw(0)

    async def load_button(self, button_type):
        if self.df.shape[0] == 0:
            return
        seq = self.df.iloc[0]["seq"]
        print("current first seq %d" % seq)
        if button_type == "left":
            seq -= self.df_cache_size / 2
        elif button_type == "right":
            seq += self.df_cache_size / 2
        await self.load_from_seq(seq)


    def handle_start_date_change(self, change):
        self.queue.put_nowait({"type": "start_date", "value": change.new})

    def handle_slider_change(self, change):
        self.draw(change.new)

    def btn_right_eventhandler(self, obj):
        self.queue.put_nowait({"type": "right"})

    def btn_left_eventhandler(self, obj):
        self.queue.put_nowait({"type": "left"})

async def loader(queue, ui):
    while True:
        command = await queue.get()
        queue.task_done()
        print(f'loader got {command}')
        if command["type"] == "right":
            await ui.load_button("right")
        elif command["type"] == "left":
            await ui.load_button("left")
        elif command["type"] == "start_date":
            await ui.load_from_time(command["value"])
        else:
            print("unknown command")

async def main(queue, ui):

    # fire up the both producers and consumers
    asyncio.create_task(loader(queue, ui))

queue = asyncio.Queue()
ui = DemoUI(queue)


#asyncio.run()
await main(queue, ui)


loader got {'type': 'start_date', 'value': datetime.date(2022, 8, 31)}
stime 2022-08-31T00:00:00.000000Z
nats_stream_fetch 10
loaded time 2022-08-31 15:08:30.285935 seq 34
loaded time 2022-08-31 15:08:39.264129 seq 35
loaded time 2022-08-31 15:08:48.239826 seq 36
loaded time 2022-08-31 15:08:57.216570 seq 37
loaded time 2022-08-31 15:09:06.193766 seq 38
loaded time 2022-08-31 15:09:15.170574 seq 39
loaded time 2022-08-31 15:09:24.149816 seq 40
loaded time 2022-08-31 15:09:33.125098 seq 41
loaded time 2022-08-31 15:09:42.101143 seq 42
loaded time 2022-08-31 15:09:51.078573 seq 43
draw 0
length of samples 3900
